In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
df = pd.read_csv('data.csv',encoding='cp949')
df['날짜'] = pd.to_datetime(df['날짜'])
df = df.fillna(0)
df.rename(columns={'날짜':'Date','오존':'Ozone', '이산화질소':'NO2','일산화탄소':'CO','아황산가스':'SO2','평균기온':'Temp','강수량':'Rain','풍속':'Wind_Speed','풍향':'Wind_Direction','상대습도':'Humid'},inplace=True)
df.drop(['PM2.5'],axis=1, inplace=True)
df

먼저 PM10와 다른 지표들과의 correlation 분석

In [ ]:
corr_matrix=df.corr()
corr_matrix["PM10"].sort_values(ascending =False)

correlation 막대 그래프로 시각화

In [ ]:
corr_df=pd.DataFrame(corr_matrix["PM10"].sort_values(ascending=False))

plt.figure(figsize=(12,8))
plt.bar(corr_df.index,corr_df["PM10"])
plt.xticks(rotation=45)

Heatmap으로 시각화

In [ ]:
corr=df.corr()
print(corr)

sns.heatmap(corr,cmap='viridis')
sns.set(rc = {'figure.figsize':(20,20)})
plt.show()

Scatter Matrix로 시각화

In [ ]:
from pandas.plotting import scatter_matrix

attributes=['Ozone','NO2','CO','SO2','Temp','Rain','Wind_Speed','Wind_Direction','Humid','PM10']

scatter_matrix(df[attributes],figsize=(20,12))

모든 지표(독립변수)를 적용하였을때 regression 진행

In [ ]:
from sklearn.model_selection import train_test_split
x=df[['Ozone','NO2','CO','SO2','Temp','Rain','Wind_Speed','Wind_Direction','Humid']]
y=df[['PM10']]

x_train, x_test, y_train,y_test=train_test_split(x,y,train_size=0.8,test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression

mlr=LinearRegression()
mlr.fit(x_train,y_train)

정확성 30퍼센트의 지리는 모델 도출

In [ ]:
print(mlr.score(x_train,y_train))

In [ ]:
plt.plot(mlr.predict(x_test[:365]),label="predict")
plt.plot(y_test[:365].values.reshape(-1,1), label="real")
plt.legend()

왜 이런 정확도 30퍼의 모델이 되었냐??
그래서 1차적으로 각 독립변수의 유의성을 판단하여 유의하지 않은 지표 제거

각 독립변수의 유의성 판단

In [ ]:
x1=sm.add_constant(x,has_constant='add')
multi_model=sm.OLS(y,x1)
fitted_multi_model=multi_model.fit()
fitted_multi_model.summary()

P value를 통해 유의수준 95퍼센트에서 검증 결과 SO2, 강수량과 풍향을 제외시킬 수 있었다. 

In [ ]:
df.drop(['SO2','Rain','Wind_Direction'],axis=1, inplace=True)
df

2차로 무얼하냐?
다중공선성 측정(vif(variance inflation factor)가 10이상인것 제외 시킨다)
however 없으니 다중공선성은 걍 넘어간다

In [ ]:
x_new=df[['Ozone','NO2','CO','Temp','Wind_Speed','Humid']]
x_new1=sm.add_constant(x_new, has_constant="add")

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(x_new1.values,i) for i in range(x_new1.shape[1])]
vif['features']=x_new1.columns
vif

그렇다면 다시 다중회귀분석을 해볼까?

In [ ]:
xx=x_new
yy=y
xx_train,xx_test,yy_train,yy_test=train_test_split(xx,yy,train_size=0.8, test_size=0.2)

In [ ]:
mlr1=LinearRegression()
mlr1.fit(xx_train,yy_train)

In [ ]:
print(mlr1.score(xx_train,yy_train))

In [ ]:
print(mlr1.coef_)

In [ ]:
multi_model2=sm.OLS(y,x_new1)
fitted_multi_model2=multi_model2.fit()
fitted_multi_model2.summary()